In [2]:
from db_manager.AugurInterface import AugurInterface
import pandas as pd

In [3]:
augur_db = AugurInterface()
augur_db.load_pconfig([
    "cali",
    "!baseball21",
    "chaoss.tv",
    "5432",
    "padres",
    "augur_data",
])
engine = augur_db.get_engine()
if engine is None:
    print("No Engine.")

In [4]:
repo = 36109 

query_string = f"""
                    SELECT
                        --r.repo_name,
                        --c.cmt_commit_hash AS commits,
                        --c.cmt_id AS file,
                        --c.cmt_added AS lines_added,
                        --c.cmt_removed AS lines_removed,
                        --c.cmt_committer_date AS date,
                        c.cmt_committer_timestamp as timestamp
                    FROM
                        repo r
                    JOIN commits c
                    ON r.repo_id = c.repo_id
                    WHERE
                        c.repo_id = {repo}
                    order by timestamp asc
                    """

In [5]:
df_commits = augur_db.run_query(query_string)
df_commits["posix_ts_int"] = pd.to_datetime(df_commits["timestamp"], utc=True).map(pd.Timestamp.timestamp).astype('int64')
print(df_commits.head())

                   timestamp  posix_ts_int
0  2005-04-16 17:20:36-05:00    1113690036
1  2005-04-16 17:20:36-05:00    1113690036
2  2005-04-16 17:20:36-05:00    1113690036
3  2005-04-16 17:20:36-05:00    1113690036
4  2005-04-16 17:20:36-05:00    1113690036


In [6]:
print(df_commits.memory_usage(deep=True))
print(df_commits.dtypes)

Index                 128
timestamp       141886808
posix_ts_int     20269544
dtype: int64
timestamp       object
posix_ts_int     int64
dtype: object


In [7]:
df_commits["new_datetime"] = pd.to_datetime(df_commits["posix_ts_int"], unit="s")
print(df_commits.head())

                   timestamp  posix_ts_int        new_datetime
0  2005-04-16 17:20:36-05:00    1113690036 2005-04-16 22:20:36
1  2005-04-16 17:20:36-05:00    1113690036 2005-04-16 22:20:36
2  2005-04-16 17:20:36-05:00    1113690036 2005-04-16 22:20:36
3  2005-04-16 17:20:36-05:00    1113690036 2005-04-16 22:20:36
4  2005-04-16 17:20:36-05:00    1113690036 2005-04-16 22:20:36


In [23]:
import datetime
by_period = pd.to_datetime(df_commits['new_datetime']).dt.to_period("M").value_counts().sort_index()
by_period.index = pd.PeriodIndex(by_period.index).to_series().astype(str)
# by_period.index = by_period.index.to_timestamp(freq="M")
#by_period.index = by_period.index.to_series().astype(str)
by_period = by_period.rename_axis('per_date').reset_index(name='counts')
# by_period["per_date"] = by_period['per_date'].apply(lambda x: x.strftime("%m/%Y"))
by_period

,per_date,counts
0,2005-04,18766
1,2005-05,3364
2,2005-06,6201
3,2005-07,4039
4,2005-08,3755
...,...,...
206,2022-06,14920
207,2022-07,18145
208,2022-08,11779
209,2022-09,15358


In [20]:
print(by_period.dtypes)

per_date    object
counts       int64
dtype: object


In [ ]:
%%timeit

df_commits.to_parquet('test_parquet.parquet', engine='pyarrow', index=False)

614 ms ± 6.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%%timeit

df_temp = pd.read_parquet('test_parquet.parquet')

20.8 ms ± 51.2 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
print(df_commits.shape)

(2533693, 3)
